# Costruzione dataset per addestramento modelli

In [ ]:
import pandas as pd
import os
from datetime import datetime
import calendar

In [ ]:
dataset_base_path = os.path.join("Dataset", "Dati")
final_dataset_path = "Dataset"

In [ ]:
def make_timestamp(field, year, month, day):
    time, ampm = field.split(" ")
    h, m, s = time.split(":")
    if h == "12":
        h = "00"
    if ampm == "PM":
        h = str(int(h)+12)
    dtime = datetime(int(year), int(month), int(day), int(h), int(m), int(s))
    return str(dtime)

In [ ]:
list_of_field = [
    "Time", "Timestamp", "Year", "Month", "Day", "Hour", "Minute", "Second",
    #"PL1 React1", "PL2 React2", "PL3 React3", 
    "P_HeatPump", 
    #"P_Cabine", 
    "P_CabinePower_L1", 
    "P_CabineLight_L3", 
    #"P_M2", 
    "P L1 WS Emergency Light", "P L2 WS Emergency Light", "P L3 WS Emergency Light", "P WS Emergency Light", 
    "P L1 WS Light", "P L2 WS Light", "P L3 WS Light", "P WS Light", 
    "P L1 WS FanCoil", "P L2 WS FanCoil", "P L3 WS FanCoil", "P WS FanCoil", 
    "P L1 WS FM", "P L2 WS FM", "P L3 WS FM", "P WS FM"       
]

In [ ]:
def missing_days(y, m, list_of_days):
    tmp = calendar.monthrange(y, m)[1]
    total_days = set(range(1,tmp+1))
    list_of_days = set(list_of_days)
    return total_days.difference(list_of_days)

In [ ]:
check_file_shape = {}
check_dir = {}

df = pd.DataFrame()
for dir_name in os.listdir(dataset_base_path):
    
    dir_files = os.path.join(dataset_base_path, dir_name)
    days = []
    for filename in os.listdir(dir_files):
        
        # filename = Plant_F40_2021_06_01
        plant, f40, year, month, day = filename.split(".")[0].split("_")
        
        days.append(int(day))
        
        file_path = os.path.join(dir_files, filename)
        print(file_path)
        
        df_tmp = pd.read_csv(file_path, delimiter="\t")
        df_tmp["Time"] = df_tmp["Time"].apply(lambda x: make_timestamp(x, year, month, day))
        
        # "2011-02-02 11:00:00"
        date_format = "%Y-%m-%d %H:%M:%S"
        df_tmp["Timestamp"] = df_tmp["Time"].apply(lambda x: datetime.strptime(x, date_format).timestamp())
        df_tmp["Year"] = df_tmp["Time"].apply(lambda x: x.split(" ")[0].split("-")[0])
        df_tmp["Month"] = df_tmp["Time"].apply(lambda x: x.split(" ")[0].split("-")[1])
        df_tmp["Day"] = df_tmp["Time"].apply(lambda x: x.split(" ")[0].split("-")[2])
        df_tmp["Hour"] = df_tmp["Time"].apply(lambda x: x.split(" ")[1].split(":")[0])
        df_tmp["Minute"] = df_tmp["Time"].apply(lambda x: x.split(" ")[1].split(":")[1])
        df_tmp["Second"] = df_tmp["Time"].apply(lambda x: x.split(" ")[1].split(":")[2])
        
        df_tmp = df_tmp.loc[:, list_of_field]
        
        if not df_tmp.shape == (5760, len(list_of_field)):
            check_file_shape[file_path] = df_tmp.shape
        
        df = pd.concat([df, df_tmp])

    miss_days = missing_days(int(year), int(month), days)
    if len(miss_days) > 0:
        check_dir[year+" "+month] = miss_days

In [ ]:
path = os.path.join(final_dataset_path, "dataset.csv")
df.to_csv(path, index=False)

In [ ]:
# Orario solare --> 31 ottobre 2021 (errore shape)
# Orario legale --> 27 marzo 2022 (manca file)
check_file_shape

In [ ]:
path_error_shape = "Dataset\\Dati\\2021 10\\Plant_F40_2021_10_31.csv"
df_error_shape = pd.read_csv(path_error_shape, delimiter="\t")
df_error_shape[df_error_shape.duplicated(['Time'])]

In [ ]:
check_dir

# Costruzione dataset per analisi temporale (plot)

In [ ]:
import pandas as pd
import os
from datetime import datetime
import calendar

In [ ]:
dataset_base_path = os.path.join("Dataset", "Dummy")
final_dataset_plot_path = "Dataset"

In [ ]:
def make_timestamp(field, year, month, day):
    time, ampm = field.split(" ")
    h, m, s = time.split(":")
    if h == "12":
        h = "00"
    if ampm == "PM":
        h = str(int(h)+12)
    dtime = datetime(int(year), int(month), int(day), int(h), int(m), int(s))
    return str(dtime)

In [ ]:
list_of_field = [
    "Time", "Timestamp", "Year", "Month", "Day", "Hour", "Minute", "Second",
    "P_HeatPump", "P_CabinePower_L1", "P_CabineLight_L3", 
    "P WS Emergency Light", "P WS Light", "P WS FanCoil", "P WS FM"       
]

In [ ]:
for year in range(2021,2023):
    for month in range(1,13):  
        dir_files = os.path.join(dataset_base_path, str(year)+" "+str(month).zfill(2))
        if not os.path.exists(dir_files):
            os.mkdir(dir_files)
        for day in range(1,calendar.monthrange(year, month)[1]+1):
            print(year, month, day)
            # Plant_F40_2022_03_01.csv
            filename = "Plant_F40_" + str(year) + "_" + str(month).zfill(2) + "_" + str(day).zfill(2) + ".csv"
            path = os.path.join(dir_files, filename)
            data = []
            if not os.path.exists(path):
                for hour in range(0,24):
                    for minute in range(0,60):
                        for second in [11, 26, 41, 56]:
                            dtime = datetime(year, month, day, hour, minute, second)
                            data.append({
                                "Time": str(dtime), 
                                "Timestamp": dtime.timestamp(), 
                                "Year": year, 
                                "Month": month, 
                                "Day": day, 
                                "Hour": hour, 
                                "Minute": minute, 
                                "Second": second,
                                "P_HeatPump": 0, 
                                "P_CabinePower_L1": 0, 
                                "P_CabineLight_L3": 0,
                                "P WS Emergency Light": 0, 
                                "P WS Light": 0, 
                                "P WS FanCoil": 0, 
                                "P WS FM": 0    
                            })
                df = pd.DataFrame(data)
                df.to_csv(path, index=False)
                
            else:
                df = pd.read_csv(path, delimiter="\t")
                df["Time"] = df["Time"].apply(lambda x: make_timestamp(x, year, month, day))

                # "2011-02-02 11:00:00"
                date_format = "%Y-%m-%d %H:%M:%S"
                df["Timestamp"] = df["Time"].apply(lambda x: datetime.strptime(x, date_format).timestamp())
                df["Year"] = df["Time"].apply(lambda x: x.split(" ")[0].split("-")[0])
                df["Month"] = df["Time"].apply(lambda x: x.split(" ")[0].split("-")[1])
                df["Day"] = df["Time"].apply(lambda x: x.split(" ")[0].split("-")[2])
                df["Hour"] = df["Time"].apply(lambda x: x.split(" ")[1].split(":")[0])
                df["Minute"] = df["Time"].apply(lambda x: x.split(" ")[1].split(":")[1])
                df["Second"] = df["Time"].apply(lambda x: x.split(" ")[1].split(":")[2])

                df = df.loc[:, list_of_field]
                df.to_csv(path, index=False)

In [ ]:
path = os.path.join(final_dataset_plot_path, "dummy_dataset.csv")
df = pd.DataFrame()
for dir_name in os.listdir(dataset_base_path):
    dir_files = os.path.join(dataset_base_path, dir_name)
    for filename in os.listdir(dir_files):
        print(filename)
        file_path = os.path.join(dir_files, filename)
        df_tmp = pd.read_csv(file_path)
        df = pd.concat([df, df_tmp])
df.to_csv(path, index=False)

# Costruzione dataset mediato

In [1]:
import pandas as pd
import os
from datetime import datetime
import calendar

In [2]:
dataset_base_path = os.path.join("Dataset", "Dati")
final_dataset_path = "Dataset"

In [3]:
def make_timestamp(field, year, month, day):
    time, ampm = field.split(" ")
    h, m, s = time.split(":")
    if h == "12":
        h = "00"
    if ampm == "PM":
        h = str(int(h)+12)
    dtime = datetime(int(year), int(month), int(day), int(h), int(m), int(s))
    return str(dtime)

In [37]:
def make_date(row):
    return datetime(int(row["Year"]), int(row["Month"]), int(row["Day"]), int(row["Hour"])).timestamp()

In [38]:
list_of_field = [
    "P_HeatPump", "P_CabinePower_L1", "P_CabineLight_L3", 
    "P WS Emergency Light", "P WS Light", "P WS FanCoil", "P WS FM"       
]

In [41]:
df = pd.DataFrame()
for dir_name in os.listdir(dataset_base_path):
    
    dir_files = os.path.join(dataset_base_path, dir_name)
    days = []
    for filename in os.listdir(dir_files):
        
        # filename = Plant_F40_2021_06_01
        plant, f40, year, month, day = filename.split(".")[0].split("_")
        
        days.append(int(day))
        
        file_path = os.path.join(dir_files, filename)
        print(file_path)
        
        df_tmp = pd.read_csv(file_path, delimiter="\t")
        df_tmp["Time"] = df_tmp["Time"].apply(lambda x: make_timestamp(x, year, month, day))
        
        # "2011-02-02 11:00:00"
        date_format = "%Y-%m-%d %H:%M:%S"
        df_tmp["Timestamp"] = df_tmp["Time"].apply(lambda x: datetime.strptime(x, date_format).timestamp())
        df_tmp["Year"] = df_tmp["Time"].apply(lambda x: x.split(" ")[0].split("-")[0])
        df_tmp["Month"] = df_tmp["Time"].apply(lambda x: x.split(" ")[0].split("-")[1])
        df_tmp["Day"] = df_tmp["Time"].apply(lambda x: x.split(" ")[0].split("-")[2])
        df_tmp["Hour"] = df_tmp["Time"].apply(lambda x: x.split(" ")[1].split(":")[0])
        df_tmp["Minute"] = df_tmp["Time"].apply(lambda x: x.split(" ")[1].split(":")[1])
        df_tmp["Second"] = df_tmp["Time"].apply(lambda x: x.split(" ")[1].split(":")[2])
                
        df_tmp = df_tmp.groupby(["Year", "Month", "Day", "Hour"])[list_of_field].mean()
        df_tmp = df_tmp.reset_index()
        df_tmp["Timestamp"] = df_tmp.apply(make_date, axis=1)
        
        df = pd.concat([df, df_tmp])

Dataset\Dati\2021 06\Plant_F40_2021_06_01.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_02.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_03.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_04.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_05.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_06.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_07.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_08.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_09.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_10.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_11.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_12.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_13.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_14.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_15.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_16.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_17.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_18.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_19.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_20.csv
Dataset\Dati\2021 06\Plant_F40_2021_06_21.csv
Dataset\Dati\2021 06\Plant_F40_202

Dataset\Dati\2021 12\Plant_F40_2021_12_18.csv
Dataset\Dati\2021 12\Plant_F40_2021_12_19.csv
Dataset\Dati\2021 12\Plant_F40_2021_12_20.csv
Dataset\Dati\2021 12\Plant_F40_2021_12_21.csv
Dataset\Dati\2021 12\Plant_F40_2021_12_22.csv
Dataset\Dati\2021 12\Plant_F40_2021_12_23.csv
Dataset\Dati\2021 12\Plant_F40_2021_12_24.csv
Dataset\Dati\2021 12\Plant_F40_2021_12_25.csv
Dataset\Dati\2021 12\Plant_F40_2021_12_26.csv
Dataset\Dati\2021 12\Plant_F40_2021_12_27.csv
Dataset\Dati\2021 12\Plant_F40_2021_12_28.csv
Dataset\Dati\2021 12\Plant_F40_2021_12_29.csv
Dataset\Dati\2021 12\Plant_F40_2021_12_30.csv
Dataset\Dati\2021 12\Plant_F40_2021_12_31.csv
Dataset\Dati\2022 01\Plant_F40_2022_01_01.csv
Dataset\Dati\2022 01\Plant_F40_2022_01_02.csv
Dataset\Dati\2022 01\Plant_F40_2022_01_03.csv
Dataset\Dati\2022 01\Plant_F40_2022_01_04.csv
Dataset\Dati\2022 01\Plant_F40_2022_01_05.csv
Dataset\Dati\2022 01\Plant_F40_2022_01_06.csv
Dataset\Dati\2022 01\Plant_F40_2022_01_07.csv
Dataset\Dati\2022 01\Plant_F40_202

In [42]:
df

,Year,Month,Day,Hour,P_HeatPump,P_CabinePower_L1,P_CabineLight_L3,P WS Emergency Light,P WS Light,P WS FanCoil,P WS FM,Timestamp
0,2021,06,01,00,17.977504,83.754287,5.342058,1497.041667,852.666667,0.000000,1263.666667,1.622498e+09
1,2021,06,01,01,8.967467,84.113329,5.377392,1513.708333,863.791667,0.000000,1249.208333,1.622502e+09
2,2021,06,01,02,8.813079,83.958163,5.283517,1496.291667,852.625000,0.000000,1316.791667,1.622506e+09
3,2021,06,01,03,8.801996,84.261346,4.031350,1490.708333,847.916667,0.000000,1261.291667,1.622509e+09
4,2021,06,01,04,8.902746,84.370717,1.366896,1494.500000,851.000000,0.000000,1263.125000,1.622513e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
19,2022,05,31,19,375.136483,83.660942,5.469125,1536.625000,675.250000,92.625000,3505.041667,1.654016e+09
20,2022,05,31,20,357.499563,84.192696,5.612742,1498.666667,680.541667,90.000000,2586.958333,1.654020e+09
21,2022,05,31,21,334.256617,84.676946,5.668521,1506.583333,683.208333,89.916667,2606.291667,1.654024e+09
22,2022,05,31,22,287.612071,85.116646,5.567238,1513.250000,691.500000,90.000000,2451.458333,1.654027e+09


In [43]:
path = os.path.join(final_dataset_path, "mean_dataset.csv")
df.to_csv(path, index=False)